In [6]:
from transformers import AutoModel, AutoTokenizer, AutoConfig
from datasets import load_dataset
import torch
import random
import numpy as np

seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

do_random = True
model_name = "microsoft/deberta-v3-xsmall"
ds_name = "amazon_polarity"
device = "cuda:1"
dtype = torch.float16
tokenizer = AutoTokenizer.from_pretrained(model_name)
if do_random:
    config = AutoConfig.from_pretrained(model_name)
    model = AutoModel.from_config(config).to(device).to(dtype)
else:
    model = AutoModel.from_pretrained(model_name).to(device).to(dtype)
ds_dict = load_dataset(ds_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/mnt/ssd-1/alexm/miniconda3/envs/ql/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
from datasets import DatasetDict
# ds_dict = DatasetDict({"train": ds_dict["train"].select(range(16)), "test": ds_dict["test"].select(range(16))})
ds_dict = DatasetDict({"train": ds_dict["train"].select(range(3_600_000)), "test": ds_dict["test"].select(range(10_000))})
ds_dict

DatasetDict({
    train: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 3600000
    })
    test: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 10000
    })
})

In [8]:
import torch
from datasets import Features, Sequence, Value, ClassLabel

model.eval()
def get_last_hidden_state(exs: dict):
    encodings = tokenizer(exs["content"], truncation=True, padding=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**encodings)
        exs["embedding"] = outputs.last_hidden_state.squeeze(1)[:, 0, :].cpu().float()  # batch_size x embedding_size
    return exs

ds = ds_dict.with_format("torch")
feats = Features({"content": Value("string"), "label": ClassLabel(num_classes=2, names=["neg", "pos"]), "embedding": Sequence(Value("float32")), "title": Value("string")})
ds = ds.map(get_last_hidden_state, batched=True, batch_size=128, features=feats)

In [ ]:
name = "atmallen/amazon_polarity_embeddings"
if do_random:
    name += f"_random{seed}"
ds.push_to_hub(name)

Creating parquet from Arrow format: 100%|██████████| 240/240 [00:03<00:00, 61.97ba/s]/it]


Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  6.00it/s]


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X = ds["train"]["embedding"]
y = ds["train"]["label"]
print(y.mean(dtype=torch.float32))
clf = LogisticRegression(random_state=0).fit(X, y)
y_pred = clf.predict(ds["test"]["embedding"])
print(accuracy_score(ds["test"]["label"], y_pred))

tensor(0.5000)
0.7244


/mnt/ssd-1/alexm/miniconda3/envs/ql/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
